In [ ]:
import os
import torch
import timm
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import seaborn as sns
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn, optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from timm.data import resolve_model_data_config
print("Section runs successfully")

In [ ]:
CFG = {
    # give corresponding model name from model finder given at the end of the notebook
    'model_name': 'swinv2_base_window8_256.ms_in1k',
    # give your corresponding batch size
    'batch_size': 32,
    # give corresponding epoch number
    'epochs': 50,
    # give appropriate learning rate
    'learning_rate': 1e-5,
    # num of workers from 0-4
    'num_workers': 2,
    # seed for reproducibility
    'seed': 42,
    # adjust your training directory
    'train_directory': '/kaggle/input/insulator-images/Insulator images/Dataset/train',
    # adjust your validation directory
    'val_directory': '/kaggle/input/insulator-images/Insulator images/Dataset/validation',
    # adjust your test directory
    'test_directory': '/kaggle/input/insulator-images/Insulator images/Dataset/test',
    'save_path': './model_checkpoints',
    'results_dir': './results'
}
os.makedirs(CFG['save_path'], exist_ok=True)
os.makedirs(CFG['results_dir'], exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

print("Section runs successfully")

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(CFG['seed'])

print("Section runs successfully")

In [ ]:
data_config = resolve_model_data_config(CFG['model_name'])
print("\n",data_config)

print("Section runs successfully")

In [ ]:
from timm.data import resolve_model_data_config
from torchvision import transforms

# Model + Data Transforms
# ----------------------------
def get_model(model_name, num_classes, device=torch.device("cpu")):
    model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)
    return model.to(device)



def get_data_transforms(model, is_training=False):
    data_config = resolve_model_data_config(model)
    #print("\n",data_config)
    input_size = data_config.get('input_size', (3, 224, 224))[1:]
    mean = data_config.get('mean', (0.485, 0.456, 0.406))
    std = data_config.get('std', (0.229, 0.224, 0.225))

    print(f"\n[INFO] Creating {'training' if is_training else 'validation/testing'} transform")
    print(f"[INFO] Using input size: {input_size}, mean: {mean}, std: {std}")

    transform_list = []
    if is_training:
        transform_list += [
            transforms.RandomResizedCrop(input_size),
            transforms.RandomHorizontalFlip(),
            transforms.RandAugment(num_ops=2, magnitude=9)
        ]
    else:
        transform_list += [
            transforms.Resize([int(x / 0.95) for x in input_size]),
            transforms.CenterCrop(input_size)
        ]

    transform_list += [
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ]

    return transforms.Compose(transform_list)


print("Section runs successfully")

In [ ]:
from torchvision import datasets
from torch.utils.data import DataLoader

# ----------------------------
# Data Loaders
# ----------------------------

def prepare_dataloader(directory, model, batch_size, is_training):
    transform = get_data_transforms(model, is_training=is_training)
    dataset = datasets.ImageFolder(directory, transform=transform)

    if is_training:
        # Ensures reproducible shuffling in training
        generator = torch.Generator()
        generator.manual_seed(CFG['seed'])
        loader = DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=CFG['num_workers'],
            pin_memory=True,
            generator=generator
        )
    else:
        loader = DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=CFG['num_workers'],
            pin_memory=True
        )

    return loader, dataset.classes


def get_dataloaders():
    # Temporary model for transform config
    temp_model = get_model(CFG['model_name'], num_classes=1, device=device)
    
    # Load training data once to get class names
    train_loader, class_names = prepare_dataloader(
        CFG['train_directory'], temp_model, CFG['batch_size'], is_training=True
    )

    # Redefine model with correct num_classes
    num_classes = len(class_names)
    model = get_model(CFG['model_name'], num_classes, device)

    # Use the final model for val/test dataloaders
    val_loader, _ = prepare_dataloader(
        CFG['val_directory'], model, CFG['batch_size'], is_training=False)
    test_loader, _ = prepare_dataloader(
        CFG['test_directory'], model, CFG['batch_size'], is_training=False)

    return train_loader, val_loader, test_loader, class_names


print("Section runs successfully")

In [ ]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, correct = 0.0, 0
    for images, labels in tqdm(loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * images.size(0)
        correct += (outputs.argmax(dim=1) == labels).sum().item()
    return total_loss / len(loader.dataset), correct / len(loader.dataset)

def validate(model, loader, criterion, device):
    model.eval()
    total_loss, correct = 0.0, 0
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)
            correct += (outputs.argmax(dim=1) == labels).sum().item()
    return total_loss / len(loader.dataset), correct / len(loader.dataset)


print("Section runs successfully")

In [ ]:
import pandas as pd

def run_training():
    train_loader, val_loader, test_loader, class_names = get_dataloaders()
    model = get_model(CFG['model_name'], len(class_names), device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=CFG['learning_rate'])

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    best_val_acc = 0.0
    best_model_state = None

    for epoch in range(1, CFG['epochs'] + 1):
        print(f"\nEpoch {epoch}/{CFG['epochs']}")
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate(model, val_loader, criterion, device)

        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Val   Loss: {val_loss:.4f}, Val   Acc: {val_acc:.4f}")

        # Save best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }
            best_path = os.path.join(CFG['save_path'], f"best_model__{CFG['model_name']}.pt")
            torch.save(best_model_state, best_path)
            print(f"[✔] Best model updated and saved at epoch {epoch} -> {best_path}")

        # Record metrics
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)

    # Save last epoch model
    last_model_state = {
        'epoch': CFG['epochs'],
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    last_path = os.path.join(CFG['save_path'], f"last_model_epoch_{CFG['epochs']}__{CFG['model_name']}.pt")
    torch.save(last_model_state, last_path)
    print(f"[✔] Last epoch model saved at: {last_path}")

    # Plot and save metrics
    plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies)

    # Save training log to Excel
    metrics_df = pd.DataFrame({
        'Epoch': list(range(1, CFG['epochs'] + 1)),
        'Train Loss': train_losses,
        'Val Loss': val_losses,
        'Train Accuracy': train_accuracies,
        'Val Accuracy': val_accuracies
    })

    log_path = os.path.join(CFG['results_dir'], f"epoch_metrics_{CFG['model_name']}.xlsx")
    metrics_df.to_excel(log_path, index=False)
    print(f"[✔] Training log saved to: {log_path}")


In [ ]:


def plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies):
    epochs = list(range(1, len(train_losses) + 1))

    # A base width plus an increment for each epoch.
    # You can tune these values for better visual results.
    base_width = 8 # inches
    width_per_epoch = 0.4 # inches per epoch
    dynamic_width = max(base_width, len(epochs) * width_per_epoch)

    # Loss Plot
    plt.figure(figsize=(dynamic_width, 6)) # Height is fixed at 6 inches, width is dynamic
    plt.plot(epochs, train_losses, 'bo-', label='Train Loss')  # blue circle markers
    plt.plot(epochs, val_losses, 'rs-', label='Val Loss')      # red square markers
    plt.xticks(epochs)  # Force integer ticks on x-axis
    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.tight_layout() # Adjust layout to prevent labels from overlapping
    plt.savefig(os.path.join(CFG['results_dir'], f"loss_curve_{CFG['model_name']}.png"))
    plt.show()
    plt.close()

    # Accuracy Plot
    plt.figure(figsize=(dynamic_width, 6)) # Apply the same dynamic width
    plt.plot(epochs, train_accuracies, 'g^-', label='Train Acc')  # green triangle markers
    plt.plot(epochs, val_accuracies, 'md-', label='Val Acc')      # magenta diamond markers
    plt.xticks(epochs)  # Force integer ticks on x-axis
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid()
    plt.tight_layout() # Adjust layout to prevent labels from overlapping
    plt.savefig(os.path.join(CFG['results_dir'], f"accuracy_curve_{CFG['model_name']}.png"))
    plt.show()
    plt.close()

print("Section runs successfully")

In [ ]:
def load_model(model_name, num_classes, epoch, path):
    model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)
    checkpoint = torch.load(f"{path}/model_epoch_{epoch}__{CFG['model_name']}.pt", map_location='cpu',weights_only=True)
    model.load_state_dict(checkpoint['model_state_dict'])  # ✅ only load the weights
    model.eval()
    return model


print("Section runs successfully")

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings  # Just use warnings
from sklearn.metrics import classification_report

# Ignore UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UserWarning)  # Filter UserWarnings in general

def plot_confusion_matrix(conf_matrix, class_names, model_name, save_dir='./results'):
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    
    # Save figure
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"confusion_matrix_{model_name}.png")
    plt.savefig(save_path)
    print(f"Confusion matrix saved to: {save_path}")
    
    plt.show()
    plt.close()


def evaluate_and_save(model, dataloader, class_names, save_dir):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    all_preds = []
    all_labels = []
   
    model.to(device)

    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Metrics
    acc = accuracy_score(all_labels, all_preds)
    
    # Using zero_division=1 to handle undefined metrics
    report = classification_report(all_labels, all_preds, target_names=class_names, digits=5, output_dict=True, zero_division=1)

    conf_matrix = confusion_matrix(all_labels, all_preds)

    print(f"\n Test Accuracy: {acc:.4f}\n")
    print("Classification Report:\n", classification_report(all_labels, all_preds, digits=5, target_names=class_names))
    print("Confusion Matrix:\n", conf_matrix)

    # Plot Confusion Matrix
    plot_confusion_matrix(conf_matrix, class_names, CFG['model_name'], save_dir)


    # Save to Excel
    os.makedirs(save_dir, exist_ok=True)
    report_df = pd.DataFrame(report).transpose()
    conf_df = pd.DataFrame(conf_matrix, index=class_names, columns=class_names)

    with pd.ExcelWriter(f"{save_dir}/Test_Metrics_{CFG['model_name']}.xlsx") as writer:
        report_df.to_excel(writer, sheet_name='Classification_Report')
        conf_df.to_excel(writer, sheet_name='Confusion_Matrix')

    print(f"\nResults saved to: {save_dir}/test_metrics.xlsx")

    return acc, report_df, conf_df
    

print("Section runs successfully")

In [ ]:
run_training()

In [ ]:
# Load dataloaders
train_loader, val_loader, test_loader, class_names = get_dataloaders()

# --------- Evaluate Best Model ---------
best_model_path = os.path.join(CFG['save_path'], f"best_model__{CFG['model_name']}.pt")
best_model = get_model(CFG['model_name'], len(class_names), device)
checkpoint = torch.load(best_model_path, map_location=device)
best_model.load_state_dict(checkpoint['model_state_dict'])

print("\n[🔍] Evaluating Best Model...")
acc_best, report_best, conf_best = evaluate_and_save(best_model, test_loader, class_names, save_dir='./results/best_model')
print(f"[🏆] Best Model Testing Accuracy: {acc_best:.4f}")

In [ ]:
# --------- Evaluate Last Model ---------
last_model_path = os.path.join(CFG['save_path'], f"last_model_epoch_{CFG['epochs']}__{CFG['model_name']}.pt")
last_model = get_model(CFG['model_name'], len(class_names), device)
checkpoint = torch.load(last_model_path, map_location=device)
last_model.load_state_dict(checkpoint['model_state_dict'])

print("\n[🔍] Evaluating Last Epoch Model...")
acc_last, report_last, conf_last = evaluate_and_save(last_model, test_loader, class_names, save_dir='./results/last_model')
print(f"[📦] Last Epoch Model Testing Accuracy: {acc_last:.4f}")

print("✅ Section ran successfully.")

In [ ]:
avail_pretrained_models = timm.list_models('*swin*',pretrained=True)
print(len(avail_pretrained_models))
avail_pretrained_models